<a href="https://colab.research.google.com/github/amiranissian/ai-ml-principles-exercises/blob/main/ML-training/logging-trainings-with-WandB/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies and import packages
First we need to install the libraries we will be using. We will use `numpy` for generic matrix operations and `tensorflow` for deep learning operations such as convolutions, pooling and training (backpropagation).

In [1]:
import sys
!{sys.executable} -m pip install numpy tensorflow
from IPython.display import clear_output
clear_output()
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

## Create a neural network
Next we define a function that can be used to build a neural network. The neural network is a simple CNN (convolutional neural network) used for classification. The structure of the network is not important for this exercise, you can instead see it as a black box that can be trained to classify an input image.

In [2]:
def create_cnn(input_shape, output_classes):
    return keras.Sequential(
        [
            keras.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
            layers.MaxPooling2D(pool_size=(2, 2)),
            layers.Flatten(),
            layers.Dense(output_classes, activation="softmax"),
        ]
    )

## Prepare the data
The neural network will be trained on a digit classification dataset called *MNIST*. This code downloads and loads the images together with their true labels. The code also does some preprocessing of the data to make it more suitable for a neural network.

In [3]:
def get_mnist_data():
    # Load the data and split it between train and test sets
    (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

    # Scale images to the [0, 1] range
    x_train = x_train.astype("float32") / 255
    x_test = x_test.astype("float32") / 255

    # Make sure images have shape (28, 28, 1)
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)

    # convert class vectors to binary class matrices
    num_classes = 10
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)

    return (x_train, y_train), (x_test, y_test)

## Train the network
Finally we will train the network on the data to teach it how to classify a digit. We create a model which expects a 28x28 pixel monocolor image since this is the format the images in the *MNIST* dataset are. We then create an optimizer and calls the `fit()` method to start the training.

In [7]:
batch_size = 128
epochs = 2

In [8]:
# Get the training data
x_train, y_train = get_mnist_data()[0]

In [9]:
# Create a Convolutional Neural Network that
# expects a 28x28 pixel image with 1 color chanel (gray) as input
model = create_cnn((28, 28, 1), 10)

model.compile(loss="categorical_crossentropy",
              optimizer="adam", metrics=["accuracy"])
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1)

Epoch 1/2
422/422 [==============================] - 41s 96ms/step - loss: 0.2931 - accuracy: 0.9178 - val_loss: 0.0757 - val_accuracy: 0.9802
Epoch 2/2
422/422 [==============================] - 40s 96ms/step - loss: 0.0745 - accuracy: 0.9778 - val_loss: 0.0570 - val_accuracy: 0.9837


In [11]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 7.9 MB/s 
     |████████████████████████████████| 168 kB 68.8 MB/s 
     |████████████████████████████████| 182 kB 56.6 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 168 kB 69.4 MB/s 
     |████████████████████████████████| 166 kB 74.9 MB/s 
     |████████████████████████████████| 166 kB 61.0 MB/s 
     |████████████████████████████████| 162 kB 59.9 MB/s 
     |████████████████████████████████| 162 kB 57.1 MB/s 
     |████████████████████████████████| 158 kB 60.2 MB/s 
     |████████████████████████████████| 157 kB 46.4 MB/s 
     |████████████████████████████████| 157 kB 60.7 MB/s 
     |████████████████████████████████| 157 kB 57.4 MB/s 
     |████████████████████████████████| 157 kB 61.2 MB/s 
     |████████████████████████████████| 157 kB 58.2 MB/s 
     |████████████████████████████

In [12]:
# Log in to your W&B account
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [16]:
batch_size = 128
epochs = 3
wandb.init(project="digit_classification_sklearn.ipynb",
           config={"batch_size": batch_size, "epochs": epochs})

In [17]:
model.fit(x_train, y_train, batch_size=batch_size,
          epochs=epochs, validation_split=0.1,
          callbacks=[wandb.keras.WandbCallback()])
wandb.finish()

Epoch 1/3
422/422 [==============================] - ETA: 0s - loss: 0.0317 - accuracy: 0.9899

wandb: Adding directory to artifact (/content/wandb/run-20221207_200636-342wn3bj/files/model-best)... Done. 0.0s


422/422 [==============================] - 44s 104ms/step - loss: 0.0317 - accuracy: 0.9899 - val_loss: 0.0411 - val_accuracy: 0.9893
Epoch 2/3
422/422 [==============================] - 42s 99ms/step - loss: 0.0278 - accuracy: 0.9912 - val_loss: 0.0420 - val_accuracy: 0.9880
Epoch 3/3
422/422 [==============================] - 41s 98ms/step - loss: 0.0235 - accuracy: 0.9925 - val_loss: 0.0422 - val_accuracy: 0.9900


accuracy,▁▄█
epoch,▁▅█
loss,█▅▁
val_accuracy,▆▁█
val_loss,▁▇█
accuracy,0.99248
best_epoch,0
best_val_loss,0.04107
epoch,2
loss,0.02352
val_accuracy,0.99
